In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Demographic Data Processing


In [19]:
#this data is for 11 variables, all census tracts in California, using ACS 2009 5YR estimates 

demog_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/demographic_data/'
#get percentages of each race
race = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B02001_race_clean.csv')
race["Percent White"] = race["Estimate; White alone"]/race["Estimate; Total:"]
race["Percent Black or African African"] = race["Estimate; Black or African American alone"]/race["Estimate; Total:"]
race["Percent Asian"] = race["Estimate; Asian alone"]/race["Estimate; Total:"]
#race["Tract"] = (race["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


edu = pd.read_csv(demog_data_path+ 'ACS_09_5YR_S1501_bach_clean.csv')
edu = edu.rename({"Percent of population 25 years and over  with Bachelor\'s degree": "Percent of population 25 years and over with Bachelor\'s degree"}, axis=1)
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].astype(float).dropna()


#get rid of + and ***
hvalue = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25077_hvalue_clean.csv')
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float).dropna()

#add attached and detached and find the %
single = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25024_singlefam_clean.csv')
single["Total Single Family"] = single["Estimate; 1, attached"]+ single["Estimate; 1, detached"]
single["% Single Family"] = single["Total Single Family"] / single["Estimate; Total:"]
#single["Tract"] = (single["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



#find % Hispanic and non hispanic
hispanic = pd.read_csv(demog_data_path+ 'ACS_17_5YR_B03003_hispanic_clean.csv')
hispanic["% Hispanic or Latino"] = hispanic["Estimate; Total: - Hispanic or Latino"]/hispanic["Estimate; Total:"]
hispanic["% not Hispanic or Latino"] = hispanic["Estimate; Total: - Not Hispanic or Latino"]/hispanic["Estimate; Total:"]
#hispanic["Tract"] = (hispanic["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



poverty = pd.read_csv(demog_data_path+ "ACS_17_5YR_S1701_poverty_clean.csv")
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].astype(float).dropna()
# poverty["Tract"] = (poverty["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

units = pd.read_csv(demog_data_path+'ACS_09_5YR_B25001_units_clean.csv')
#units["Tract"] = (units["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


#find % renter occupied
tenure = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25003_tenure_clean.csv')
tenure["% Renter Occupied"] = tenure["Estimate; Renter occupied"]/tenure["Estimate; Total:"]
tenure["% Owner Occupied"] = tenure["Estimate; Owner occupied"]/tenure["Estimate; Total:"]
#tenure["Tract"] = (tenure["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


income = pd.read_csv(demog_data_path + 'ACS_09_5YR_S1903_income.csv')
#income["Tract"] = (income["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

ValueError: could not convert string to float: '1,000,000'

In [8]:
# nonzeros = np.nonzero(['-' in x for x in edu["Percent of population 25 years and over  with Bachelor\'s degree"]])
# [edu["Percent of population 25 years and over  with Bachelor\'s degree"][i] for i in nonzeros]
# edu["Percent of population 25 years and over  with Bachelor\'s degree"][1902] == '-'
edu["Percent of population 25 years and over with Bachelor\'s degree"].dtype

dtype('float64')

#### Create Controlled features dataframe

In [171]:
race_units_merged = race.merge(units, how='left', left_on="Id2", right_on="Id2")
print("race and units merged", "length", len(race_units_merged))
edu_merged = race_units_merged.merge(edu, how='left', left_on="Id2", right_on="Id2")
print("edu merged", "length", len(edu_merged))
hispanic_merged = edu_merged.merge(hispanic, how='left', left_on="Id2", right_on="Id2")
print("hispanic merged", "length", len(hispanic_merged))
income_merged = hispanic_merged.merge(income, how='left', left_on="Id2", right_on="Id2")
print("income merged", "length", len(income_merged))
poverty_merged = income_merged.merge(poverty, how='left', left_on="Id2", right_on="Id2")
print("poverty merged", "length", len(poverty_merged))
hvalue_merged = poverty_merged.merge(hvalue, how='left', left_on="Id2", right_on="Id2")
print("hvalue merged", "length", len(hvalue_merged))

single_merged = hvalue_merged.merge(single, how='left', left_on="Id2", right_on="Id2")
print("single merged", "length", len(single_merged))

merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
print("tenure merged", "length", len(merged))

race and units merged length 7049
edu merged length 7049
hispanic merged length 7049
income merged length 7049
poverty merged length 7049
hvalue merged length 7049
single merged length 7049
tenure merged length 7049


In [172]:
merged.columns
merged = merged[["Id2", 'Percent of population 25 years and over  with Bachelor\'s degree', '% Hispanic or Latino', 
      'Percent below poverty level; Estimate; Population for whom poverty status is determined', 
      '% Single Family', '% Owner Occupied', 'Percent Black or African African', 'Percent White', 'Percent Asian',
      'Estimate; Total Number of Housing Units', 'Estimate; Median value (dollars)', 
        'Median income (dollars); All households']]
merged = merged.rename({'Percent below poverty level; Estimate; Population for whom poverty status is determined': "% below poverty level",
              'Estimate; Total Number of Housing Units': 'Total number of housing units',
              'Estimate; Median value (dollars)': "Median home value",
              'Median income (dollars); All households': "Median income"})
      
merged.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,"1,000,000",186439
1,6001400200,37.6,0.088633,5.5,0.665236,0.657428,0.022640,0.794798,0.069364,932,909500,122647
2,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100,66638
3,6001400400,44,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500,80391
4,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000,50658


#### Load in and merge with neighbors Dataframe

In [173]:
neighbors_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/'
boolean_neighbors = pd.read_csv(neighbors_data_path+ 'boolean_tracts_all_2009.csv')
#boolean_neighbors["Tract"] = boolean_neighbors["Tract"].astype(int)
boolean_neighbors.head()

,Tract,Year,CRA Eligible
0,6001433700,2009,eligible
1,6001435101,2009,not eligible
2,6001435101,2009,not eligible
3,6001435101,2009,not eligible
4,6001435101,2009,not eligible


In [174]:
demog_with_cra = merged.merge(boolean_neighbors, how='right', left_on='Id2', right_on='Tract').drop("Id2", axis=1) #i think that this is supposed to
#be a right merge because the demographics dataframe has all the census tracts, while the boolean neighbors one only
#has the census tracts in the bay area that have neighbors
demog_with_cra.head()

,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,Tract,Year,CRA Eligible
0,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,"1,000,000",186439,6001400100,2009,not eligible
1,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100,66638,6001400300,2009,not eligible
2,44,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500,80391,6001400400,2009,not eligible
3,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000,50658,6001400500,2009,eligible
4,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700,39802,6001400600,2009,eligible


#### Get Loans Data

In [175]:
filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data/'
Alameda = pd.read_csv(filepath+'Alameda_2009_parsed.csv')
Alameda["Geoid"] = Alameda["Tract"].apply(lambda x: int(x*100 + 6075000000))

ContraCosta = pd.read_csv(filepath+'ContraCosta_2009_parsed.csv')
ContraCosta["Geoid"] = ContraCosta["Tract"].apply(lambda x: int(x*100 + 6075000000))

Napa = pd.read_csv(filepath+'Napa_2009_parsed.csv')
Napa["Geoid"] = Napa["Tract"].apply(lambda x: int(x*100 + 6075000000))

Marin = pd.read_csv(filepath+'Marin_2009_parsed.csv')
Marin["Geoid"] = Marin["Tract"].apply(lambda x: int(x*100 + 6075000000))

SanMateo = pd.read_csv(filepath+'SanMateo_2009_parsed.csv')
SanMateo["Geoid"] = SanMateo["Tract"].apply(lambda x: int(x*100 + 6075000000))

SanFrancisco = pd.read_csv(filepath+'SanFrancisco_2009_parsed.csv')
SanFrancisco["Geoid"] = SanFrancisco["Tract"].apply(lambda x: int(x*100 + 6075000000))

Solano = pd.read_csv(filepath+'Solano_2009_parsed.csv')
Solano["Geoid"] = Solano["Tract"].apply(lambda x: int(x*100 + 6075000000))

SantaClara = pd.read_csv(filepath+'SantaClara_2009_parsed.csv')
SantaClara["Geoid"] = SantaClara["Tract"].apply(lambda x: int(x*100 + 6075000000))

Sonoma = pd.read_csv(filepath+'Sonoma_2009_parsed.csv')
Sonoma["Geoid"] = Sonoma["Tract"].apply(lambda x: int(x*100 + 6075000000))

counties = [Alameda, ContraCosta, Napa, Marin, SanMateo, SanFrancisco,Solano,SantaClara, Sonoma  ]

all_counties_loans = pd.concat(counties).drop("Unnamed: 0", axis=1)
all_counties_loans["Total Loans"] = all_counties_loans['# HI borrower, LI tract'] + all_counties_loans['# HI borrower, HI tract']+ all_counties_loans['# LI borrower, LI tract']
all_counties_loans['# LI borrower, HI tract']
all_counties_loans.head()



,Tract,Year,County,type,CRA Eligible,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract","# LI borrower, LI tract","%LI borrower, HI tract","# LI borrower, HI tract",Geoid,Total Loans
0,4001.0,2009,Alameda County,high,not eligible,0.000000,0,0.981818,54,0.000000,0,0.018182,1,6075400100,54
1,4002.0,2009,Alameda County,high,not eligible,0.000000,0,1.000000,20,0.000000,0,0.000000,0,6075400200,20
2,4003.0,2009,Alameda County,mid,not eligible,0.000000,0,0.906250,58,0.000000,0,0.093750,6,6075400300,58
3,4004.0,2009,Alameda County,mid,not eligible,0.000000,0,0.979167,47,0.000000,0,0.020833,1,6075400400,47
4,4005.0,2009,Alameda County,mod,eligible,0.877193,50,0.000000,0,0.122807,7,0.000000,0,6075400500,57


#### Merge Loans Data with cra_demog features

In [176]:
allvars_2009 = all_counties_loans.merge(demog_with_cra, how="right", left_on='Geoid', right_on='Tract').drop(["Tract_x", "Year_x", "County", 'CRA Eligible_y', 'Tract_y', 'Year_y'], axis=1)
allvars_2009['CRA'] = allvars_2009['CRA Eligible_x'].apply(lambda x: 1 if x == 'eligible' else 0)
#allvars_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars2009.csv")



#### Regression

In [177]:
allvars_2009.columns
noinc = ['% HI borrower, LI tract',
       '# HI borrower, LI tract', '% HI borrower, HI tract',
       '# HI borrower, HI tract', '% LI borrower, LI tract',
       '# LI borrower, LI tract', '%LI borrower, HI tract',
       '# LI borrower, HI tract', 'Geoid', 'Total Loans', "CRA Eligible_x", "type"]
X_vars = allvars_2009.loc[:, ~allvars_2009.columns.isin(noinc)]
y = allvars_2009["Total Loans"]
X_vars.head()

,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,39.7,0.070078,13.5,0.039784,0.136743,0.164179,0.546269,0.262415,2036,720200,50368,1
1,41.5,0.030713,5.8,0.090069,0.346681,0.010335,0.816491,0.088884,3031,"1,000,000",89722,0
2,37,0.092529,10,0.124668,0.300904,0.022752,0.626874,0.316916,2262,"1,000,000",82118,0
3,44.4,0.117330,6.6,0.061102,0.300522,0.000000,0.703414,0.268467,2995,855300,74850,0
4,36.5,0.106652,19.4,0.077562,0.268998,0.004555,0.778054,0.157764,1805,947500,119290,0


In [114]:
def normalize_columns(data, mean_df, std_df):
    '''
    Input:
      data (data frame): contains only numeric columns
    Output:
      data frame, the same data, except each column is standardized 
      to have 0-mean and unit variance
    '''
    normalized_data=(data-mean_df.mean())/std_df.std()

    return normalized_data

In [115]:
def unnormalize_columns(data, mean_df, std_df):
    unnormalized_data=(data*std_df.std())+mean_df.mean()

    return unnormalized_data

In [118]:
# normal_x_vars = normalize_columns(X_vars, X_vars, X_vars)
# normal_y = normalize_columns(y, y, y)

In [178]:
X_vars.dtypes

Percent of population 25 years and over  with Bachelor's degree                             object
% Hispanic or Latino                                                                       float64
Percent below poverty level; Estimate; Population for whom poverty status is determined     object
% Single Family                                                                            float64
% Owner Occupied                                                                           float64
Percent Black or African African                                                           float64
Percent White                                                                              float64
Percent Asian                                                                              float64
Estimate; Total Number of Housing Units                                                      int64
Estimate; Median value (dollars)                                                            object
Median inc

In [119]:
from sklearn import linear_model
sk_model = linear_model.LinearRegression()
sk_model.fit(X_vars,y)


ValueError: could not convert string to float: '-'